# Algoritmul HillClimbing in Python
Pentru problema rucsacului

In [1]:
import numpy as np

In [2]:
def calculeaza_calitate(selectie, costuri, valori, capacitate_maxima):
    """
    Verifică fezabilitatea unei soluții și calculează valoarea funcției obiectiv.

    Parametri:
    - selectie: Lista binară (0/1) reprezentând obiectele selectate
    - costuri: Lista costurilor asociate obiectelor
    - valori: Lista valorilor asociate obiectelor
    - capacitate_maxima: Capacitatea maximă a rucsacului

    Returnează:
    - este_fezabil: Boolean (True dacă soluția este fezabilă, False în caz contrar)
    - valoare_totala: Suma valorilor obiectelor selectate
    """
    cost_total = np.dot(selectie, costuri)
    valoare_totala = np.dot(selectie, valori)

    return cost_total <= capacitate_maxima, valoare_totala

In [3]:
def genereaza_vecini(selectie_curenta, costuri, valori, capacitate_maxima):
    """
    Calculează toți vecinii fezabili ai unei soluții și calitățile lor.

    Un vecin este obținut prin modificarea stării unui singur obiect
    (adăugare sau eliminare).

    Parametri:
    - selectie_curenta: Lista binară (0/1) reprezentând obiectele selectate
    - costuri: Lista costurilor asociate obiectelor
    - valori: Lista valorilor asociate obiectelor
    - capacitate_maxima: Capacitatea maximă a rucsacului

    Returnează:
    - lista_vecini: Matrice de vecini fezabili
    - calitati_vecini: Lista valorilor funcției obiectiv pentru fiecare vecin
    """
    n = selectie_curenta.size
    lista_vecini = np.zeros(0, dtype="int")
    calitati_vecini = np.zeros(0, dtype="float")

    for i in range(n):
        vecin = selectie_curenta.copy()
        vecin[i] = not selectie_curenta[i]  # Schimbă starea unui singur obiect (0->1 sau 1->0)

        este_fezabil, valoare = calculeaza_calitate(vecin, costuri, valori, capacitate_maxima)

        if este_fezabil:
            lista_vecini = np.append(lista_vecini, vecin)
            calitati_vecini = np.append(calitati_vecini, valoare)

    # Reorganizează vectorul de vecini într-o matrice
    if len(lista_vecini) > 0:
        dim = len(lista_vecini)
        lista_vecini = lista_vecini.reshape(round(dim/n), n)

    return lista_vecini, calitati_vecini

In [4]:
def hill_climbing_rucsac(fisier_costuri, fisier_valori, numar_iteratii, capacitate_maxima):
    """
    Implementare a algoritmului Hill Climbing pentru problema rucsacului 0-1.

    Parametri:
    - fisier_costuri: Calea către fișierul cu costurile obiectelor
    - fisier_valori: Calea către fișierul cu valorile obiectelor
    - numar_iteratii: Numărul de execuții independente ale algoritmului
    - capacitate_maxima: Capacitatea maximă a rucsacului

    Returnează:
    - solutie_optima: Lista binară reprezentând cea mai bună selecție de obiecte găsită
    - valoare_maxima: Valoarea maximă obținută
    - toate_solutiile: Toate soluțiile locale găsite în fiecare execuție
    - toate_valorile: Valorile corespunzătoare fiecărei soluții locale

    Exemple de apel:
    sol, val, P, C = hill_climbing_rucsac("cost.txt", "valoare.txt", 70, 50)  # max 81
    sol, val, P, C = hill_climbing_rucsac("cost1.txt", "valoare1.txt", 90, 50)  # max 108
    sol, val, P, C = hill_climbing_rucsac("cost2.txt", "valoare2.txt", 1000, 56.6)  # max 128.2
    """
    # Citirea datelor din fișiere
    costuri = np.genfromtxt(fisier_costuri)
    valori = np.genfromtxt(fisier_valori)
    n = costuri.size  # Numărul de obiecte

    # Inițializare structuri pentru stocarea rezultatelor
    toate_solutiile = np.zeros([numar_iteratii, n], dtype="int")
    toate_valorile = np.zeros(numar_iteratii, dtype="float")

    # Execută algoritmul Hill Climbing de mai multe ori
    for iteratie in range(numar_iteratii):
        # Generează un punct de start fezabil aleator
        am_gasit_punct_start = False
        while not am_gasit_punct_start:
            selectie_curenta = np.random.randint(0, 2, n)  # Generează un vector binar aleator
            este_fezabil, valoare_curenta = calculeaza_calitate(selectie_curenta, costuri, valori, capacitate_maxima)
            am_gasit_punct_start = este_fezabil

        # Execută algoritmul Hill Climbing
        am_ajuns_la_maxim_local = False
        while not am_ajuns_la_maxim_local:
            # Generează și evaluează toți vecinii
            vecini, calitati = genereaza_vecini(selectie_curenta, costuri, valori, capacitate_maxima)

            if calitati.size == 0:
                # Nu există vecini fezabili
                am_ajuns_la_maxim_local = True
            else:
                # Găsește cel mai bun vecin
                index_vecin_maxim = np.argmax(calitati)
                vecin_maxim = vecini[index_vecin_maxim]
                valoare_vecin_maxim = calitati[index_vecin_maxim]

                if valoare_vecin_maxim > valoare_curenta:
                    # Dacă am găsit un vecin mai bun, continuăm cu el
                    selectie_curenta = vecin_maxim
                    valoare_curenta = valoare_vecin_maxim
                else:
                    # Altfel, am ajuns la un maxim local
                    am_ajuns_la_maxim_local = True

        # Salvează soluția locală găsită
        toate_solutiile[iteratie] = selectie_curenta.copy()
        toate_valorile[iteratie] = valoare_curenta

    # Identifică cea mai bună soluție găsită
    valoare_maxima = np.max(toate_valorile)
    index_maxim = np.argmax(toate_valorile)
    solutie_optima = toate_solutiile[index_maxim]

    print("Cea mai bună valoare calculată:", valoare_maxima)
    print("Selecția optimă de obiecte:", solutie_optima)

    return solutie_optima, valoare_maxima, toate_solutiile, toate_valorile

# exemplu apel
solutie1, valoare1, toate_solutiile1, toate_valorile1 = hill_climbing_rucsac("cost.txt", "valoare.txt", 70, 50)

Cea mai bună valoare calculată: 80.0
Selecția optimă de obiecte: [0 0 0 0 0 0 1 1 1 1 0 0 0 0 1]
